# Classify Images via Transfer Learning

We will build a simple model in Cloud ML using a small set of labeled flower images. This dataset has been selected for ease of explanation only; We've successfully used the same implementation for several proprietary datasets covering cases like interior-design classification (e.g., carpet vs. hardwood floor) and animated-character classification. The code can be found https://github.com/GoogleCloudPlatform/cloudml-samples/tree/master/flowers and can easily be adapted to run on different datasets.

## Preprocess Data
We start with a set of labeled images in a Google Cloud Storage bucket and preprocess them to extract the image features from the bottleneck layer (typically the penultimate layer) of the Inception network. Although processing images in this manner can be reasonably expensive, each image can be processed independently and in parallel, making this task a great candidate for Cloud Dataflow.

We process each image to produce its feature representation (also known as an embedding) in the form of a k-dimensional vector of floats (in our case 2,048 dimensions). The preprocessing includes converting the image format, resizing images, and running the converted image through a pre-trained model to get the embeddings. Final output will be written in directory specified by --output_path.

In [1]:
import time
GCS_BUCKET = 'gs://ml-workshop-transfer-learning' #CHANGE THIS TO YOUR BUCKET
PROJECT = 'march-ml-workshop' #CHANGE THIS TO YOUR PROJECT ID
REGION = 'us-central1' #OPTIONALLY CHANGE THIS
TIME = str(int(time.time()))
GCS_PATH= GCS_BUCKET + "/" + TIME

import os
os.environ['GCS_BUCKET'] = GCS_BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['GCS_PATH'] = GCS_PATH

In [2]:
!gsutil mb $GCS_BUCKET
!gsutil cp -r gs://cloud-ml-data/img/flower_photos/train_set.csv $GCS_BUCKET/img/flower_photos/
!gsutil cp -r gs://cloud-ml-data/img/flower_photos/eval_set.csv $GCS_BUCKET/img/flower_photos/
!gsutil cp -r gs://cloud-ml-data/img/flower_photos/dict.txt $GCS_BUCKET/img/flower_photos/



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Creating gs://ml-workshop-transfer-learning/...
Copying gs://cloud-ml-data/img/flower_photos/train_set.csv [Content-Type=text/csv]...
/ [1 files][260.0 KiB/260.0 KiB]                                                
Operation completed over 1 objects/260.0 KiB.                                    
Copying gs://cloud-ml-data/img/flower_photos/eval_set.csv [Content-Type=text/csv]...
/ [1 files][ 29.3 KiB/ 29.3 KiB]                                                
Operation completed over 1 objects/29.3 KiB.                                     
Copying gs://cloud-ml-data/img/flower_photos/dict.txt [Content-Type=text/plain]...
/ [1 files][   40.0 B/   40.0 B]                                                
Operation completed over 1 objects/40.0 B.                                       


In [3]:
!gsutil cat -r 0-85  $GCS_BUCKET/img/flower_photos/eval_set.csv
!gsutil cat $GCS_BUCKET/img/flower_photos/dict.txt

gs://cloud-ml-data/img/flower_photos/sunflowers/14646281372_5f13794b47.jpg,sunflowers
daisy
dandelion
roses
sunflowers
tulips


In [4]:
!gsutil cat gs://cloud-ml-data/img/flower_photos/train_set.csv | wc -l
!gsutil cat gs://cloud-ml-data/img/flower_photos/eval_set.csv | wc -l

3300
370


### Machine Learning Pipeline 
We are setting up following pipeline 
![Machine Learning Pipeline](pipeline.png)

<br>
<br>
<br>
# SKIP THIS STEP & Go to Next 

<br>
<br>
<br>
### Preprocessing 
(Optional: This take about 50 minutes to run in DataFlow over 20 single core VMs)
<br><br>
We start with a set of labeled images in a Google Cloud Storage bucket and preprocess them to extract the image features from the bottleneck layer (typically the penultimate layer) of the Inception network. Although processing images in this manner can be reasonably expensive, each image can be processed independently and in parallel, making this task a great candidate for Cloud Dataflow.

We process each image to produce its feature representation (also known as an embedding) in the form of a k-dimensional vector of floats (in our case 2,048 dimensions). The preprocessing includes converting the image format, resizing images, and running the converted image through a pre-trained model to get the embeddings. Final output will be written in directory specified by --output_path.

<br>
(uri, label_ids, image_bytes) -> (tensorflow.Example).

  Output proto contains 'label', 'image_uri' and 'embedding'.
  The 'embedding' is calculated by feeding image into input layer of image
  neural network and reading output of the bottleneck layer of the network.

In [ ]:
!python trainer/preprocess.py \
  --input_dict $GCS_BUCKET/img/flower_photos/dict.txt \
  --input_path $GCS_BUCKET/img/flower_photos/eval_set.csv \
  --output_path $GCS_PATH/preproc/eval \
  --num_workers 10 \
  --cloud

In [ ]:
!python trainer/preprocess.py \
  --input_dict $GCS_BUCKET/img/flower_photos/dict.txt \
  --input_path $GCS_BUCKET/img/flower_photos/train_set.csv \
  --output_path $GCS_PATH/preproc/train \
  --num_workers 10 \
  --cloud

/usr/local/lib/python2.7/dist-packages/oauth2client/contrib/gce.py:99: UserWarning: You have requested explicit scopes to be used with a GCE service account.
Using this argument will have no effect on the actual scopes for tokens
requested. These scopes are set at VM instance creation time and
can't be overridden in the request.

  warnings.warn(_SCOPES_WARNING)
/usr/local/lib/python2.7/dist-packages/apache_beam/io/gcp/gcsio.py:113: DeprecationWarning: object() takes no parameters
  super(GcsIO, cls).__new__(cls, storage_client))
/usr/local/lib/python2.7/dist-packages/apache_beam/coders/typecoders.py:135: UserWarning: Using fallback coder for typehint: Any.
  warnings.warn('Using fallback coder for typehint: %r.' % typehint)
DEPRECATION: pip install --download has been deprecated and will be removed in the future. Pip now has a download command that should be used instead.
    100% |████████████████████████████████| 583kB 1.7MB/s 
  Saved /tmp/tmpB1fDK8/google-cloud-dataflow-2.0.0.tar.

### Preprocessed Images: Simple copy tfrecords below (Avoid 50 min. Processing time)

In [ ]:
!gsutil -m cp gs://lytx-experiment/1512512700/preproc/* $GCS_PATH/preproc/

### Training
Once we've preprocessed data, we can then train a simple classifier. The network will comprise a single fully-connected layer with RELU activations and with one output for each label in the dictionary to replace the original output layer. Final output is computed using the softmax function


![Training](incept_v3.png)

In [ ]:
%%bash
echo $GCS_PATH
echo $GCS_BUCKET

In [ ]:
%%bash
JOBNAME=lytx_$(date -u +%y%m%d_%H%M%S)
gcloud ml-engine jobs submit training $JOBNAME \
  --stream-logs \
  --module-name=trainer.task \
  --package-path=./trainer \
  --staging-bucket=$GCS_BUCKET \
  --region=us-central1 \
  --runtime-version=1.0 \
  --scale-tier=STANDARD_1 \
  -- \
  --output_path=$GCS_PATH/$JOBNAME/output \
  --eval_data_paths=$GCS_PATH/preproc/eval* \
  --train_data_paths=$GCS_PATH/preproc/train*

In [31]:
%%bash
echo $JOBNAME

## TensorBoard - View our Training Progress

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://lytx-dry-run/1513014343/lytx_171211_182542/output')

In [ ]:
%%bash
MODEL_NAME="lytx_dry_run"
MODEL_VERSION="v1"
MODEL_LOCATION="gs://lytx-dry-run/1513014343/lytx_171211_182542/output/model" #REPLACE this with the location of your model
gcloud ml-engine models create ${MODEL_NAME} --regions us-central1
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} 

In [ ]:
!gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg daisy.jpg

In [ ]:
%%bash
python -c 'import base64, sys, json; img = base64.b64encode(open(sys.argv[1], "rb").read()); print json.dumps({"key":"0", "image_bytes": {"b64": img}})' daisy.jpg &> request.json

In [ ]:
%%bash
gcloud ml-engine predict --model lytx_prep --json-instances request.json

In [ ]:
!gsutil cat $GCS_BUCKET/img/flower_photos/dict.txt

In [ ]:
%%bash
wget https://www.heirloomroses.com/media/catalog/product/cache/1/image/650x/040ec09b1e35df139433887a97daa66f/g/r/gr842-love-1_1.jpg
mv gr842-love-1_1.jpg rose.jpg

In [ ]:
%%bash
python -c 'import base64, sys, json; img = base64.b64encode(open(sys.argv[1], "rb").read()); print json.dumps({"key":"1", "image_bytes": {"b64": img}})' rose.jpg &>> request.json

In [ ]:
%%bash
gcloud ml-engine predict --model lytx_prep --json-instances request.json